<a href="https://colab.research.google.com/github/kittisaktaoma/colab_practice/blob/master/tmp/caox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
import pandas as pd

In [52]:
#! git clone https://github.com/Superzchen/iFeature
! python iFeature/iFeature.py --file drive/My\ Drive/sample_test.fasta --type AAC --out AAC.txt

Descriptor type: AAC


In [ ]:
df = pd.read_csv("AAC.txt",sep="\t")
df2 = df.rename({'#': 'id'}, axis='columns')
df_02 = pd.DataFrame(df2.id.str.split('|',1).tolist(),columns = ['A','B'])
df_03 = pd.DataFrame(df_02.B.str.split('|',1).tolist(),columns = ['id','B'])
df_c = pd.concat([df_03.iloc[:,0], df.iloc[:,1:21]], axis=1)
df_c

In [96]:
index_df = pd.read_csv("https://raw.githubusercontent.com/kittisaktaoma/colab_practice/master/tmp/sample.txt",sep="\t")
cdf = pd.merge(df_c,index_df,on='id')
cdf

,id,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y,cat
0,P02768,0.103448,0.057471,0.059113,0.101806,0.057471,0.021346,0.026273,0.014778,0.098522,0.105090,0.011494,0.027915,0.039409,0.032841,0.044335,0.045977,0.047619,0.070608,0.003284,0.031199,1
1,P02760,0.059659,0.045455,0.034091,0.079545,0.039773,0.102273,0.011364,0.042614,0.051136,0.076705,0.028409,0.039773,0.051136,0.036932,0.053977,0.056818,0.071023,0.062500,0.014205,0.042614,1
2,P00450,0.047887,0.014085,0.062911,0.076056,0.048826,0.072300,0.038498,0.054460,0.061972,0.069484,0.023474,0.049765,0.046948,0.030986,0.037559,0.057277,0.070423,0.059155,0.017840,0.060094,1
3,P02751,0.040775,0.025434,0.050868,0.058539,0.021801,0.083973,0.020589,0.048849,0.031490,0.054905,0.010900,0.040775,0.078321,0.053290,0.051272,0.080743,0.108599,0.080743,0.016149,0.041986,1
4,P02788,0.091549,0.046479,0.053521,0.057746,0.045070,0.078873,0.012676,0.022535,0.064789,0.092958,0.008451,0.045070,0.049296,0.040845,0.061972,0.070423,0.043662,0.070423,0.014085,0.029577,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,P42566,0.072545,0.012277,0.077009,0.092634,0.046875,0.042411,0.013393,0.036830,0.066964,0.099330,0.012277,0.045759,0.071429,0.050223,0.027902,0.108259,0.051339,0.054688,0.007812,0.010045,0
155,P13612,0.058140,0.024225,0.048450,0.064922,0.043605,0.074612,0.025194,0.062016,0.055233,0.087209,0.021318,0.058140,0.042636,0.037791,0.044574,0.082364,0.049419,0.073643,0.008721,0.037791,0
156,Q9H4M9,0.063670,0.001873,0.069288,0.074906,0.050562,0.063670,0.028090,0.059925,0.082397,0.108614,0.028090,0.039326,0.059925,0.037453,0.052434,0.048689,0.031835,0.067416,0.011236,0.020599,0
157,P49619,0.049305,0.035398,0.060683,0.068268,0.035398,0.074589,0.031606,0.042984,0.067004,0.092288,0.035398,0.039191,0.058154,0.037927,0.051833,0.067004,0.050569,0.058154,0.015171,0.029077,0
